To play slideshow, run this command in the terminal:
```
$ jupyter nbconvert intro-to-testing-presentation.ipynb --to slides --post serve --SlidesExporter.reveal_theme=night --SlidesExporter.reveal_transition=none
```

# The Role of Testing in Data Science

Jes Ford, PhD

Data Scientist

<img src="img/pytest.png">

# About Me

- Originally from Alaska, have followed the snow all around the western US/Canada
- PhD in Astrophysics from UBC, Vancouver
- Postdoc in Data Science at UW, Seattle
- Moved to UT to snowboard and be a Data Scientist at Backountry.com $\rightarrow$ now at Recursion Pharma
- I love teaching and learning about things including Python
- I organize this PyLadies chapter

<!--- <table><tr>
<td> <img src="https://jesford.github.io/photos/cfht_and_me.jpg" alt="Drawing" style="width: 200px;"/> </td>
<td> <img src="https://jesford.github.io/photos/Jess_Ford_SSS_cbox_BSlide.jpg" alt="Drawing" style="width: 142px;"/> </td>
<td> <img src="https://jesford.github.io/photos/toki_lions.jpg" alt="Drawing" style="width: 315px;"/> </td>
</tr></table> --->

## Why talk about testing?

I care about code quality...

meaning clean code\* that gives *correct results*

\* topic for another day: PEP8, code is read much more often than it is written!

## Why test?

- Tests can give you evidence that your code is working as expected
- Tests give you confidence to make changes without fear of breaking something
- Tests make other people trust your code more
- ... however bad tests can give you false confidence

## Why *not* test?

- Writing tests takes time!
- As a data scientist I am constantly struggling with these competing goals:

  - getting results as quickly as possible
  - being as confident as possible that I've got the right answer
  
$\rightarrow$ How do we balance these interests in the optimal way?

## In this talk...


- I will *not* insist that you always write tests

- I will describe different scenarios I find myself in as a data scientist and how I try to be confident that my results are correct

- I will show you how to get started testing 

## Disclaimer

- I am not a testing expert or a software engineer

- These *opinions* are based on my own experience as a data scientist

- "data science" covers a huge range of job duties and formal testing is less important in some of them (one-off analyses vs committing to production code base)

## How do you know if your code is correct??
- manual sanity checks
- defensive programming: assertions within the code
- tests

In [1]:
# assertion example
def hello_to_all(list_of_names):
    assert len(list_of_names) > 0, 'There is no one here'
    print('Hello {}!'.format(', '.join(list_of_names)))

In [2]:
hello_to_all(['Ayla', 'Missy', 'Evan'])

Hello Ayla, Missy, Evan!


In [3]:
hello_to_all([])

AssertionError: There is no one here

## Simple test example

In [4]:
def backwards_allcaps(text):
    return text[::-1].upper()

In [5]:
backwards_allcaps('PyLadies')

'SEIDALYP'

In [6]:
def test_backwards_allcaps():
    assert backwards_allcaps('python') == 'NOHTYP'
    assert backwards_allcaps('meetup') == 'PUTEEM'

## Types of tests

- unit tests
- integration tests
- system tests

## When to write tests

- when you write new code
- when you make a change to your code
- when you find a bug

## Test Driven Development

TDD: write the test *before* you write the code it is testing.

- reasons to use TDD:
  - makes you define your code requirements up front
  - helps frame how you'll write the code
  - is more fun
- reasons to not use TDD:
  - not always possible to define requirements up front (data exploratoration)
  - time constraints (need quick bugfix)
  - legacy code

## pytest is great

- less boilerplate $\rightarrow$ easier/faster test writing
- automatically handles finding, collecting, running, and evaluating your tests
- when tests fail you can get a lot of useful info
- fixtures give you power and flexibility (more on this later)
- just works (with benefits) on existing tests written for unittest or nose

vs unittest requires tests to be wrapped inside classes which subclass from unittest.TestCase; pytest you just write functions with simple regular assert statements, which easier to read/write


## Demo #1

What you need:

- terminal where you can run python
- pytest library: `pip install pytest`
- any code editor/IDE
- open file `demo_tdd_intro.py`
  - you can copy-paste from GitHub or clone the repo
  - https://github.com/jesford/testing-in-data-science

## Demo #1

We learned how to:
- write simple tests with pytest
- do test driven development (TDD)
- use pytest fixtures to parametrize our tests

## More about fixtures

- keeps execution of a test separate from setup/teardown of anything thats needed for the test, so the test function itself is as simple and clear as possible.
- can have scope (e.g. session), so can create a database with schemas once per run of test suite, and then tear down afterward


## Where do tests go?

pytest searches all directories below the current directory for files that start or end with "test" (`test_*.py`, `*_test.py`) and runs any functions and classes like `def test_the_things()` and `class TestStuff()`.

```
myproject/
    myproject/
        myproject.py
        utils.py
        __init__.py
    tests/
        test_myproject.py
    setup.py
    README.md
    LICENSE.txt
```

Above is a typical directory layout, but its not required. You can tell pytest where to look for tests, so really you *can* put them pretty much anywhere.

## Data Science Scenarios
1. "One-off analysis"
2. Exploratory
3. Well defined problem

<ol start="4">
  <li>Legacy code: you inherit a large amount of legacy code written by a predecessor that will need to be maintained and potentially updated over time.</li>
</ol>

## Data Science Scenarios
1. "One-off analysis"
2. **Exploratory**  $\leftarrow$
3. Well defined problem
4. **Legacy code**  $\leftarrow$

Number 3 is simple - you should write tests as you develop your code.

But the other cases often put you in a **legacy code scenario** where you have a bunch of code without any tests! Once you realize you will need to reuse code, you should start adding tests whenever you modify it.

## Demo #2

Working with legacy code... even your own! Legacy code = any existing shared or re-used code without tests.

## Wrap up

- data scientists should not *always* write tests
- any reused or shared piece of code should probably be tested
- strive for a balance between speed and confidence in your results

### Some aspects of data science code are really hard to test!

- ML results? probabilistic outcomes?
- Think about testing properties of your data
  - distributions, missing data, expected features and datatypes

### Cool related projects to be aware of

- [pytest-xdist](https://docs.pytest.org/en/3.0.1/xdist.html) plugin for pytest so you can run tests faster in parallel
- [pytest-cov](https://pytest-cov.readthedocs.io/en/latest/index.html) plugin for measuring test coverage
- [engarde](https://engarde.readthedocs.io/en/latest/index.html) for defensive data analysis with pandas
- [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) for property-based testing and testing your code with many inputs and edge cases

### Resources & Credits

- **General testing resources**
  - Ned Batchelder's [Getting Started Testing](https://www.youtube.com/watch?v=FxSsnHeWQBY) from PyCon 2014
  - Andreas Pelme's [Introduction to pytest](https://www.youtube.com/watch?v=LdVJj65ikRY) from EuroPython 2014
  - Mark Vousden's [Python testing](https://www.youtube.com/channel/UCKaKhMyhboLoMwmeF9yxg9w) 3-part series of youtube videos
  - Justin Crown's ["WHAT IS THIS MESS?" - Writing tests for pre-existing code bases](https://www.youtube.com/watch?v=LDdUuoI_lIg) from PyCon 2017
  

- **Data Science specific resources**
  - Trey Causey's [Testing for Data Scientists](https://www.youtube.com/watch?v=GEqM9uJi64Q) from PyData Seattle 2015
  - Eric Ma's [Best Testing Practice's for Data Science](https://www.youtube.com/watch?v=yACtdj1_IxE) from PyCon 2017, with GitHub tutorial notebooks [here](https://github.com/ericmjl/data-testing-tutorial)

## Questions?